In [1]:
#!usr/bin/env python3
import pandas as pd
from pathlib import Path
import praw
from tqdm.notebook import tqdm
import pandas as pd
import yfinance as yf
import re

# Options

In [2]:
root = Path.cwd()
compressed_data_root = root / 'posts'
reddit_csv_path = root / '10y_reddit_data.csv.zst'
stock_csv_path = root / '10y_stock_data.csv.zst'

years=10

subreddits = ["stocks", "StockMarket", "investing", "wallstreetbets", "options", "trading"]

stock_dict = {
    'nvidia': ['nvda', 'nvidia'],
    'tesla': ['tsla', 'tesla'],
    'apple': ['aapl', 'apple'],
    'amazon': ['amzn', 'amazon'],
    'microsoft': ['msft', 'microsoft'],
    'google': ['googl', 'google', 'alphabet']
}

tickers = ['nvda', 'tsla', 'aapl', 'amzn', 'msft', 'googl']

praw_api = praw.Reddit(
            client_id="5uFqCBUPadVnIxKHG0hnhw",
            client_secret="LDFyai0bjEAkEQqo5joU7PjSjtq2eQ",
            user_agent="TeslaScraper:v1.0 (by u/RecognitionSame5433)"
        )

# Reddit data fields
- INDEX: Day normalized timestamp
- id
- created_utc
- author
- title
- selftext
- score
- num_comments
- query (only for api data)
- stock
- subreddit
- source (api or archive)

# Stock data fields per ticker
- INDEX: Day normalized timestamp
- Open
- High
- Low
- Close
- Volume
- Open_pct
- High_pct
- Low_pct
- Close_pct
- *Revenue
- *Earnings
- *Revenue_pct
- *Earnings_pct

In [ ]:
class RedditData:
    def __init__(self, years, subreddits, stock_dict, praw_api):
        self.subreddits = subreddits
        self.stock_dict = stock_dict
        self.api = praw_api
        self.timedelta = pd.Timedelta(days=365 * years)
        self.df = pd.DataFrame()

    def search_subreddit(self, subreddit_name:str, query:str):
        subreddit = self.api.subreddit(subreddit_name)
        gen = subreddit.search(query, sort='new', time_filter='year', limit=1000)
        rows = []
        for s in gen:
            rows.append({
                    "id": s.id,
                    "created_utc" : s.created_utc,
                    "author": str(s.author) if s.author else None,
                    "title": s.title,
                    "selftext": s.selftext,
                    "score": s.score,
                    "num_comments": s.num_comments,
                })
        return pd.DataFrame(rows)

    # Get year of data for all subreddits and queries
    def load_api_data(self):
        df_list = []
        all_query_pairs = [(stock, query)
                           for stock, query_list in self.stock_dict.items()
                           for query in query_list
                           ]
        for stock, query in tqdm(all_query_pairs, desc='Queries', position=0):
            for sub in tqdm(self.subreddits,
                            desc='Subreddits',
                            leave=False,
                            position=1):
                df = self.search_subreddit(sub, query)
                df['timestamp'] = (
                    pd.to_datetime(df['created_utc'], unit='s', utc=True)
                    .dt.normalize()
                )
                df = df.set_index('timestamp')
                df['query'] = query
                df['stock'] = stock
                df['subreddit'] = sub
                df_list.append(df)
        nonempty_dfs = [df for df in df_list if not df.empty]
        print(f'{len(nonempty_dfs)} out of {len(df_list)} queries had hits')
        new_df = pd.concat(df_list)
        new_df['query'] = ''
        self.add_data(new_df, 'api')
        print('API data loaded')

    # Parses all compressed data root
    def load_compressed_ndjson(self, root: Path):
        df_list = []
        for file_path in root.rglob('*.zst'):
            print(f'Extracting {file_path.name}')
            subreddit = file_path.stem.removesuffix('_submissions')
            chunk_iter = pd.read_json(
                file_path,
                lines=True,
                compression='zstd',
                chunksize=2**16
            )
            df_chunks = []
            # Process and append each chunk
            for df_chunk in tqdm(chunk_iter,
                                desc='File chunks processed',
                                leave=False):
                df_chunk['timestamp'] = (
                    pd.to_datetime(df_chunk['created_utc'], unit='s', utc=True)
                    .dt.normalize()
                )
                df_chunk = df_chunk.set_index('timestamp')
                df_chunk = self.restrict_time_for(df_chunk)
                df_chunk = self.restrict_columns_for(df_chunk)
                df_chunk = self.keyword_filter_for(df_chunk)
                df_chunks.append(df_chunk)
            if not df_chunks:
                raise RuntimeError('File yielded no rows')
            df = pd.concat(df_chunks)
            df = self.drop_duplicates_for(df)
            df['subreddit'] = subreddit
            df_list.append(df)
        if not df_list:
            raise RuntimeError('Failed to load any data from files')
        new_df = pd.concat(df_list)
        self.add_data(new_df, 'archive')
        print('Compressed data loaded')
    
    # Remove duplicates and check for overlap when necessary
    def add_data(self, df, source):
        df = self.drop_duplicates_for(df)
        df = self.drop_invalid_posts_for(df)
        df['source'] = source
        if self.df.empty:
            self.df = df
        else:
            new_df = pd.concat([self.df, df])
            overlap = new_df.duplicated(subset=['id', 'stock']).sum()
            if overlap == 0:
                raise ValueError('Time ranges do not overlap')
            print(f'Dataframe overlap of {overlap} rows')
            self.df = self.drop_duplicates_for(new_df).sort_index()

    def keyword_filter_for(self, df):
        keyword_cols = ['title', 'selftext']
        text_df = df[keyword_cols]
        df_list = []
        for stock, query_list in self.stock_dict.items():
            pattern = '|'.join(re.escape(q) for q in query_list)
            mask = (
                text_df
                .apply(lambda col: col.astype(str).str.contains(pattern, case=False, na=False))
                .any(axis=1)
            )
            df_part = df[mask].copy()
            df_part['stock'] = stock
            df_list.append(df_part)
        if df_list:
            new_df = pd.concat(df_list)
            new_df = self.drop_duplicates_for(new_df)
        else:
            new_df = df.iloc[0:0].copy()
        return new_df
    
    def drop_invalid_posts_for(self, df):
        keywords = ['[removed]', '[deleted]']
        pattern = '|'.join([re.escape(s) for s in keywords])
        mask = (
            df.selftext.str.contains(pattern) |
            df.title.str.contains(pattern)
        )
        new_df = df[~mask]
        return new_df
            
    def restrict_columns_for(self, df):
        reddit_columns = ['id', 'created_utc', 'author', 'title', 'selftext', 'score', 'num_comments']
        new_df = df[reddit_columns].copy()
        return new_df

    def restrict_time_for(self, df):
        if not str(df.index.dtype) == "datetime64[ns, UTC]":
            raise RuntimeError(
                f'Invalid index type: {str(df.index.dtype)}')
        start_timestamp = pd.Timestamp.utcnow() - self.timedelta
        new_df = df.sort_index().loc[start_timestamp:]
        return new_df

    # Remove duplicates for (id, stock) and enforce timeframe
    def drop_duplicates_for(self, df):
        new_df = df.drop_duplicates(subset=['id', 'stock'], inplace=False)
        return new_df


class StockData:
    def __init__(self, tickers, years, interval='1d'):
        self.tickers = tickers
        df = yf.download(tickers, period=f'{years}y', interval=interval, auto_adjust=True)
        if df is None:
            raise ValueError('Data failed to download')
        assert isinstance(df.index, pd.DatetimeIndex)
        df.index = df.index.tz_localize('UTC')
        self.df = df

    # Forward fill OHLC and zero Volume
    def impute_off_days(self):
        full_idx = pd.date_range(self.df.index.min(),
                                 self.df.index.max(),
                                 freq='D',
                                 tz='UTC')
        self.df = self.df.reindex(full_idx)
        self.df['Volume'] = self.df['Volume'].fillna(0)
        self.df = self.df.ffill()

    # Create percent change columns for prices
    def create_pct_columns(self):
        price_fields = ['Open', 'High', 'Low', 'Close']
        price_df = self.df[price_fields]
        pct_df = price_df.pct_change()
        l0_fields = pct_df.columns.get_level_values(0)
        l1_fields = pct_df.columns.get_level_values(1)
        l0_fields = [s+'_pct' for s in l0_fields]
        pct_df.columns = pd.MultiIndex.from_arrays(
            [l0_fields, l1_fields], names=pct_df.columns.names)
        self.df = pd.concat([self.df, pct_df], axis=1).sort_index(axis=1)
        self.df = self.df.drop(self.df.index.min())


def align_data(reddit_data, stock_data):
    min_date = max(reddit_data.df.index.min(), stock_data.df.index.min())
    max_date = min(reddit_data.df.index.max(), stock_data.df.index.max())
    reddit_data.df = reddit_data.df.loc[min_date:max_date]
    stock_data.df = stock_data.df.loc[min_date:max_date]

def reddit_csv():
    return pd.read_csv('10y_reddit_data.csv.zst',
                              encoding='utf-8',
                              parse_dates=True)

def stock_csv():
    return pd.read_csv('10y_stock_data.csv.zst',
                              header=[0, 1],
                              index_col=0,
                              parse_dates=True)


In [ ]:
reddit_data = RedditData(years, subreddits, stock_dict, praw_api)

In [ ]:
reddit_data.load_compressed_ndjson(compressed_data_root)

In [ ]:
reddit_data.load_api_data()

In [ ]:
print((reddit_data.df['source'] == 'api').sum())
print((reddit_data.df['source'] == 'archive').sum())
api_df = reddit_data.df[reddit_data.df['source'] == 'api']
arc_df = reddit_data.df[reddit_data.df['source'] == 'archive']

In [ ]:
print(reddit_data.df['id'].nunique())
print(api_df['id'].nunique())
print(arc_df['id'].nunique())

In [ ]:
stock_data = StockData(tickers, years)

In [ ]:
stock_data.impute_off_days()

In [ ]:
stock_data.create_pct_columns()

In [ ]:
align_data(reddit_data, stock_data)

In [ ]:
reddit_data.df.head()

In [ ]:
stock_data.df.head()

In [ ]:
stock_data.df.to_csv(stock_csv_path, encoding='utf-8')

In [ ]:
reddit_data.df.to_csv(reddit_csv_path, encoding='utf-8')

In [4]:
rdf = reddit_csv()

/tmp/ipykernel_33572/3095459165.py:197: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv('10y_reddit_data.csv.zst',


In [5]:
sdf = stock_csv()

In [6]:
rdf.head()

,timestamp,id,created_utc,author,title,selftext,score,num_comments,stock,subreddit,source,query
0,2015-12-05 00:00:00+00:00,3vj1vg,1.449312e+09,formahyi,short AMZN straddle,595/740 expiring in 40 days \n\nprimum collec...,3,27,amazon,options,archive,NaN
1,2015-12-05 00:00:00+00:00,3vk6qf,1.449337e+09,Johnny_Fratto,ITT: We make financial predictions regarding t...,Apple will hit $140/share\n\nTesla will go dow...,36,73,tesla,investing,archive,NaN
2,2015-12-05 00:00:00+00:00,3vk6qf,1.449337e+09,Johnny_Fratto,ITT: We make financial predictions regarding t...,Apple will hit $140/share\n\nTesla will go dow...,36,73,apple,investing,archive,NaN
3,2015-12-06 00:00:00+00:00,3vpco3,1.449436e+09,canoned,"Fellow investors, what are you thoughts on inv...",FB and Oculus? Google? Sony? HTC/Samsung? Othe...,74,93,google,investing,archive,NaN
4,2015-12-06 00:00:00+00:00,3vm4id,1.449369e+09,j3wbakka,YACHTpportunity in NTES,"Alright you sexy fuckers, let me hit you with ...",12,9,microsoft,wallstreetbets,archive,NaN


In [7]:
sdf.head()

Price                          Close                                   \
Ticker                          AAPL       AMZN      GOOGL       MSFT   
2015-12-05 00:00:00+00:00  26.837568  33.632000  38.693630  48.955368   
2015-12-06 00:00:00+00:00  26.837568  33.632000  38.693630  48.955368   
2015-12-07 00:00:00+00:00  26.668463  33.491501  38.384758  48.867832   
2015-12-08 00:00:00+00:00  26.657188  33.866501  38.491520  48.850327   
2015-12-09 00:00:00+00:00  26.068716  33.239498  37.866337  48.141064   

Price                                          Close_pct                      \
Ticker                         NVDA       TSLA      AAPL      AMZN     GOOGL   
2015-12-05 00:00:00+00:00  0.823295  15.358667  0.000000  0.000000  0.000000   
2015-12-06 00:00:00+00:00  0.823295  15.358667  0.000000  0.000000  0.000000   
2015-12-07 00:00:00+00:00  0.807683  15.408667 -0.006301 -0.004178 -0.007983   
2015-12-08 00:00:00+00:00  0.818661  15.114667 -0.000423  0.011197  0.002781   
2015-12-09 00:00:00+00:00  0.797926  14.968000 -0.022076 -0.018514 -0.016242   

Price                                ...  Open_pct                      \
Ticker                         MSFT  ...     GOOGL      MSFT      NVDA   
2015-12-05 00:00:00+00:00  0.000000  ...  0.000000  0.000000  0.000000   
2015-12-06 00:00:00+00:00  0.000000  ...  0.000000  0.000000  0.000000   
2015-12-07 00:00:00+00:00 -0.001788  ...  0.013446  0.030858  0.040395   
2015-12-08 00:00:00+00:00 -0.000358  ... -0.019947 -0.005736 -0.030824   
2015-12-09 00:00:00+00:00 -0.014519  ...  0.008633 -0.001803  0.025994   

Price                                     Volume                           \
Ticker                         TSLA         AAPL         AMZN       GOOGL   
2015-12-05 00:00:00+00:00  0.000000          0.0          0.0         0.0   
2015-12-06 00:00:00+00:00  0.000000          0.0          0.0         0.0   
2015-12-07 00:00:00+00:00 -0.020477  128336800.0   74656000.0  36626000.0   
2015-12-08 00:00:00+00:00 -0.000791  137238000.0   73038000.0  31082000.0   
2015-12-09 00:00:00+00:00 -0.003604  185445600.0  103164000.0  46550000.0   

Price                                                           
Ticker                           MSFT         NVDA        TSLA  
2015-12-05 00:00:00+00:00         0.0          0.0         0.0  
2015-12-06 00:00:00+00:00         0.0          0.0         0.0  
2015-12-07 00:00:00+00:00  30709800.0  333588000.0  47163000.0  
2015-12-08 00:00:00+00:00  32878000.0  314488000.0  40314000.0  
2015-12-09 00:00:00+00:00  36373200.0  372676000.0  45867000.0  

[5 rows x 54 columns]